In [1]:
import numpy as np
import torch
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse
from pathlib import Path
from pol.utils.validation.scene_saver import load_scenes, count_h5_keys, find_max_h5_key
from pol.utils.plotting import LossTrajectoryPlotter
from pol.utils.path import PathHelper

import matplotlib_inline.backend_inline
matplotlib_inline.backend_inline.set_matplotlib_formats('pdf')
import matplotlib
use_tex = matplotlib.checkdep_usetex(True)
if use_tex:
    plt.rcParams['text.usetex'] = True
plt.rcParams.update({'font.size': 28})

In [2]:
prob = 'mcp_8d'
path_helper = PathHelper('../../tests/linear_regression/')
all_satisfy = True

if prob == 'quasinorm_toy':
    all_methods = ['pd_mlr', 'pol_nvp_mot', 'pol_nvp_nm_mot', 'pol_nvp_nm_ns_mot', 'pol_res_mot']
    theta_range = range(0, 8)
    view_box = [[-0.5, 2], [-2, 0.5]]
    method_itr = {
        #'pd_mlr': 100,
        #'pol_mlp_lot': 20,
        #'pol_mlp_proj_lot': 20,
    }
    max_num_iter = 100
elif prob == 'quasinorm_8d':
    # all_methods = ['pd_lllr', 'gol_res_llr', 'pol_res_hot']
    all_methods = ['pd_hhmlr', 'pg', 'pol_res_mot']
    theta_range = range(0, 1)
    view_box = [[-2, 2], [-2, 2]]
    method_itr = {
        #'pd_mlr': 100,
        # 'pol_mlp_mot': 100,
        # 'pol_res_mot': 100,
        #'pol_mlp_proj_lot': 20,
    }
    max_num_iter = 1000
    all_satisfy = True
elif prob == 'lasso_20d':
    all_methods = ['pd_mlr', 'pol_mlp_mot', 'pol_res_mot']
    theta_range = range(0, 8)
    view_box = [[-2, 2], [-2, 2]]
    method_itr = {
        #'pd_mlr': 100,`
        #'pol_mlp_mot': 20,
        #'pol_res_mot': 20,
    }
    max_num_iter = 200
elif prob == 'mixed_8d':
    all_methods = ['pd_mlr', 'pol_res_mot', 'pol_res_lot', 'pol_res_hot']
    theta_range = range(0, 16)
    view_box = [[-2, 2], [-2, 2]]
    method_itr = {
        #'pd_mlr': 100,
        #'pol_mlp_mot': 5,
        #'pol_res_mot': 5,
        #'pol_mlp_proj_lot': 20,
    }
    max_num_iter = 1000
    all_satisfy = False
elif prob == 'mixed_ncvx_8d':
    all_methods = ['pd_lllr', 'gol_res_llr', 'pol_res_hot']
    theta_range = [0, 2, 3, 13, 1, 5, 4, 6] #  range(0, 16)
    view_box = [[-2, 2], [-2, 2]]
    method_itr = {
        #'pd_mlr': 100,
        #'pol_mlp_mot': 5,
        #'pol_res_mot': 5,
        #'pol_mlp_proj_lot': 20,
    }
    max_num_iter = 1000000
    all_satisfy = False
elif prob == 'mcp_8d':
    all_methods = ['pd_lllr', 'gol_res_llr', 'pol_res_hot', 'pg']
    theta_range = [0, 2, 5, 11] # range(0, 8)
    view_box = [[-2, 2], [-2, 2]]
    method_itr = {
        #'pd_mlr': 100,
        #'pol_mlp_mot': 5,
        #'pol_res_mot': 5,
        #'pol_mlp_proj_lot': 20,
    }
    max_num_iter = 1000000
    all_satisfy = True
else:
    assert(False)
    
cmap_base = plt.cm.get_cmap('hsv', 4)
cmap = plt.cm.get_cmap('hsv', len(all_methods)+1)

colors = {all_methods[i]: cmap_base(i) for i in range(3)}
colors.update({all_methods[i]: cmap(i) for i in range(3, len(all_methods))})
colors['pg'] = 'violet'

# colors = {all_methods[i]: cmap(i) for i in range(len(all_methods))}

def convert_method_to_label(method):
    if method.startswith('gol'):
        return 'GOL'
    if method.startswith('pol'):
        return 'POL'
    if method.startswith('pd'):
        return 'PD'
    if method.startswith('pg'):
        return 'PGD'
    return 'Unknown'

In [3]:
import ipywidgets as widgets
method_select_widget = widgets.SelectMultiple(
    options=all_methods,
    value=all_methods,
    description='Methods',
    disabled=False
)

In [4]:
loss_plotter = LossTrajectoryPlotter(
    path_helper=path_helper,
    max_num_iter=max_num_iter,
    colors=colors,
    theta_name='thetas')

def vis1():
    loss_plotter.plot_all_convergence(
        problem=prob, methods=method_select_widget.value,
        # theta_range=theta_range[:1],
        theta_range=theta_range,
        fig_size=10,
        include_hist=True,
        include_traj=True,
        satisfy_name='satisfy',
        loss_name='loss',
        loss_traj_lim=10,
        all_satisfy=all_satisfy,
        hist_max_rel=5.0,
        method_label_fn=convert_method_to_label)

In [5]:
display(method_select_widget)

SelectMultiple(description='Methods', index=(0, 1, 2, 3), options=('pd_lllr', 'gol_res_llr', 'pol_res_hot', 'p…

In [6]:
vis1()

<Figure size 2880x360 with 4 Axes>

<Figure size 2880x360 with 4 Axes>

In [7]:
def plot_particles(ax, theta_idx, methods, hide=False, print_best=False):
    def plot(X, name, color):
        ax.scatter(X[:, -2], X[:, -1], color=color, s=80.0, label=name, alpha=0.5)

    best_loss = 1e100
    for method_idx, method in enumerate(methods):
        exp_name = path_helper.format_exp_name(prob, method)
        scene = loss_plotter.load_exp_scene(prob, method)
        
        if method in method_itr:
            k = method_itr[method]
        else:
            k = find_max_h5_key(scene, 'itr', return_itr=True)
        tmp = scene['itr_{}'.format(k)]
        X = tmp['W'][theta_idx, :, :]
        
        S = tmp['satisfy'][theta_idx, :]
        
        if all_satisfy:
            S = np.ones_like(S)
        L = tmp['loss'][theta_idx, :]

        plot(X[S], convert_method_to_label(method), colors[method])
        
        if L[S].shape[0] > 0:
            L_min = L[S].min()
            if L_min < best_loss:
                best_loss = L_min
                best_X = X[S, :][L[S].argmin()]

    ax.set_xlim(view_box[0][0], view_box[0][1])
    ax.set_ylim(view_box[1][0], view_box[1][1])
    if hide:
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
        [i.set_linewidth(4.0) for i in ax.spines.values()]
    else:
        ax.legend()
        # ax.set_title(f'L_min: {best_loss}')
        # if best_loss < 1e100:
        #     print(f'theta: {theta_idx}, best_X: {best_X}')
    
def vis2():
    methods = method_select_widget.value
    fig_size = 8
    fig, axes = plt.subplots(len(methods), len(theta_range), squeeze=False)
    fig.set_figheight(fig_size*len(methods))
    fig.set_figwidth(fig_size*len(theta_range))
    for t, theta in enumerate(theta_range):
        for i, method in enumerate(methods):
            plot_particles(axes[i, t], theta, [method], hide=True)
    #fig.savefig('figs/{}_2d_vis.png'.format(prob))

In [8]:
vis2()

<Figure size 2304x2304 with 16 Axes>

In [9]:
import h5py

# h5_file = h5py.File('../../tests/linear_regression/scenes/quasinorm_8d/pol_res_hot/step-200000.h5')
h5_file = h5py.File('../../tests/linear_regression/scenes/quasinorm_8d/pg/result.h5')

FileNotFoundError: [Errno 2] Unable to open file (unable to open file: name = '../../tests/linear_regression/scenes/quasinorm_8d/pg/result.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [ ]:
scene = h5_file['scene_0']
print(scene.keys())
print(scene['itr_0'].keys())
itr_0 = scene['itr_0']
print(itr_0['loss'].shape)
print(itr_0['W'].shape)
print(itr_0['satisfy'].shape)

In [ ]:
print(scene['itr_95']['loss'][:])